In [40]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten
from keras.optimizers import Adam
import pandas as pd
from arch import arch_model
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error,mean_absolute_percentage_error, r2_score
from tensorflow import keras
from keras.wrappers.scikit_learn import KerasRegressor
from keras import backend as K

In [20]:
def read_data():
    price = pd.read_csv('dataset/kospi.csv')
    oil = pd.read_csv('dataset/oilprice.csv')
    gold = pd.read_csv('dataset/goldprice.csv')
    data = pd.DataFrame()
    data['Date'] = price['Date']
    data['Daily_trading_range'] = price['High'] - price['Low']
    data['Log_Volume_change'] = np.log((price['Volume'] / price['Volume'].shift(1))) * 100
    data['Daily_return'] = price['Close'].pct_change().dropna()
    data['Daily_log_return'] = np.log(price['Close'] / price['Close'].shift(1))
    data['Past_vol22'] = np.sqrt((data['Daily_log_return']**2)).rolling(window=22).std() * np.sqrt(252)
    data['Index'] = price['Close']
    data['gold'] = gold['Close']
    data['oil'] = oil['Close']

    volatility = np.sqrt((data['Daily_log_return'] ** 2).rolling(window=22).sum() / 22) * np.sqrt(252)
    vol10 = np.sqrt((data['Daily_log_return'] ** 2).rolling(window=10).sum() / 10) * np.sqrt(252)
    # target = yz_vol_measure(data)
    # target10 = yz_vol_measure(data, window=10)
    target22 = pd.DataFrame(volatility)
    target10 = pd.DataFrame(vol10)

    data['Target22'] = target22
    data['Target10'] = target10
    # data['Target10'] = target10
    # data = data.dropna()

    return data

In [21]:
def evaluate(predict, target, title):
    print('--------'+title+'----------')
    testScore = mean_squared_error(predict, target)
    print("test Score: {score} MSE".format(score=testScore))
    root_testScore = mean_squared_error(y_pred=predict, y_true=target, squared=False)
    print("test Score: {score} RMSE".format(score=root_testScore))
    mape = mean_absolute_percentage_error(y_pred=predict, y_true=target)
    print("test Score: {score} MAPE".format(score=mape))
    r2_test = r2_score(y_true=target, y_pred=predict)
    print("test Score: {score} R2 score".format(score=r2_test))

In [22]:
def plot(predict, target):
    plt.plot(predict, label='predict')
    plt.plot(target, label='target')
    plt.legend()
    plt.show()

In [23]:
print('split: ', split_index)

split:  1856


In [24]:
def ewma_estimation(data):
    sqreturn = np.array(data['Daily_log_return']**2)
    estimation = []
    estimation.append(data['Past_vol22'].iloc[0] / np.sqrt(252))
    weight = 0.94
    for i in range(1,len(data)):
        pred = weight*(estimation[-1]**2) + (1-weight)*(sqreturn[i-1])
        estimation.append(np.sqrt(pred))

    estimation = np.sqrt(252)*pd.DataFrame(estimation, columns=['EWMA'])

    # plot(estimation, np.array(data['Target22']))
    print('-----EWMA estimation done-----')
    return estimation

In [25]:
def create_gjr(data):
    data = data[22:].dropna()
    data_cleaned = data[22:].dropna()
    logreturns = np.array(data[['Daily_log_return']].dropna())
    target = np.array(data_cleaned[['Target22']].dropna())

    gjr_pred = []
    for i in range(len(data)):
        train = logreturns[:i + 22] * 100
        gm = arch_model(train, p=1, q=1, o=1)
        gm_fit = gm.fit(disp='off')
        pred = gm_fit.forecast(horizon=1)
        gjr_pred.append(np.sqrt(pred.variance.values[-1, :][0]) * 0.01 * np.sqrt(252))

    print('garch pred length: ', len(gjr_pred))
    print('target length: ', len(target))
    # plot(gjr_pred, target)
    gjr_pred = pd.DataFrame(gjr_pred, columns=['GJR'])
    print('-----GJR-GARCH estimation done-----')
    return gjr_pred

In [26]:
def create_garch(data):

    data = data[22:].dropna()
    data_cleaned = data[22:].dropna()
    logreturns = np.array(data[['Daily_log_return']].dropna())
    target = np.array(data_cleaned[['Target22']].dropna())

    garch_pred = []
    for i in range(len(data)):
        train = logreturns[:i+22]*100
        gm = arch_model(train, p=1, q=1)
        gm_fit = gm.fit(disp='off')
        pred = gm_fit.forecast(horizon=1)
        garch_pred.append(np.sqrt(pred.variance.values[-1,:][0])*0.01*np.sqrt(252))

    print('garch pred length: ', len(garch_pred))
    print('target length: ', len(target))
    # plot(garch_pred, target)
    garch_pred = pd.DataFrame(garch_pred, columns=['GARCH'])
    print('-----GARCH estimation done-----')
    return garch_pred

In [27]:
def data_prep(data, split_index):
    data = data.drop(['Daily_return', 'Past_vol22', 'Target10','Date'], axis=1)
    window = 22
    y_values = data[['Target22']]
    x_values = data.drop(['Target22'], axis=1)
    print(x_values.info())

    scaler = MinMaxScaler()
    scaled_x = scaler.fit_transform(x_values)
    scaled_y = scaler.fit_transform(y_values)

    trainX = np.array(scaled_x[:split_index])
    testX = np.array(scaled_x[split_index:])
    trainY = np.array(scaled_y[:split_index])
    testY = np.array(scaled_y[split_index:])

    Xtrain =[]
    ytrain =[]
    Xtest =[]
    ytest = []

    for i in range(window, len(trainX)):
        Xtrain.append(trainX[i - window:i, :trainX.shape[1]])
        ytrain.append(trainY[i])
    for i in range(window, len(testX)):
        Xtest.append(testX[i - window:i, :testX.shape[1]])
        ytest.append(testY[i])

    Xtrain, ytrain = (np.array(Xtrain), np.array(ytrain))
    Xtrain = np.reshape(Xtrain, (Xtrain.shape[0], Xtrain.shape[1], Xtrain.shape[2]))

    Xtest, ytest = (np.array(Xtest), np.array(ytest))
    Xtest = np.reshape(Xtest, (Xtest.shape[0], Xtest.shape[1], Xtest.shape[2]))

    print(Xtrain.shape)
    print(ytrain.shape)
    print("-----")
    print(Xtest.shape)
    print(ytest.shape)
    return Xtrain, ytrain, Xtest, ytest, scaler

In [39]:
def create_lstm(Xtrain, n_units=2, dropout=0):
    model = Sequential()
    model.add(LSTM(units=n_units, return_sequences=False, input_shape=(Xtrain.shape[1], Xtrain.shape[2])))
    # model.add(LSTM(units=n_units, return_sequences=True, input_shape=(Xtrain.shape[1], Xtrain.shape[2])))
    model.add(Dropout(dropout))
    # model.add(Dense(20, activation='tanh'))
    # model.add(Dense(5, activation='tanh'))
    model.add(Dense(1))
    opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)

    return model

In [29]:
def create_ann(Xtrain):
    model = Sequential()
    model.add(Flatten(input_shape=(Xtrain.shape[1], Xtrain.shape[2])))
    model.add(Dense(64, activation='tanh'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(20, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    adam = Adam(learning_rate=0.001)
    model.compile(loss='mean_squared_error', optimizer=adam)
    return model

In [30]:
def run_lstm(Xtrain, ytrain, Xtest, ytest, scaler):
    lstm = create_lstm(Xtrain)
    lstm_fit = lstm.fit(Xtrain, ytrain, batch_size=16, epochs=150)
    lstm_forecast = lstm.predict(Xtest)

    rev_forecast = scaler.inverse_transform(lstm_forecast)
    rev_ytest = scaler.inverse_transform(ytest)

    plot(rev_forecast, rev_ytest)
    evaluate(rev_forecast, rev_ytest, 'LSTM Evaluation')

In [31]:
def run_ann(Xtrain, ytrain, Xtest, ytest, scaler):
    model = create_ann(Xtrain)
    model_fit = model.fit(Xtrain, ytrain, batch_size=16, epochs=50)
    forecast = model.predict(Xtest)

    rev_forecast = scaler.inverse_transform(forecast)
    rev_ytest = scaler.inverse_transform(ytest)

    plt.plot(rev_forecast, color='red', label='forecast')
    plt.plot(rev_ytest, color='gold', label='target')
    plt.legend()
    plt.show()
    evaluate(rev_forecast, rev_ytest, 'GER-FNN evaluation')

In [32]:
data = read_data()
data_cleaned = data[22:].dropna().reset_index(drop=True)
split_index = int(len(data_cleaned)*0.8)

In [33]:
ewma_estim = ewma_estimation(data_cleaned)
garch_estim = create_garch(data)
gjr_estim = create_gjr(data)
# data_cleaned['EWMA'] = ewma_estim['EWMA']
# data_cleaned['GARCH'] = garch_estim['GARCH']
# data_cleaned['GJR'] = gjr_estim['GJR']

-----EWMA estimation done-----
garch pred length:  2321
target length:  2299
-----GARCH estimation done-----
garch pred length:  2321
target length:  2299
-----GJR-GARCH estimation done-----


/home/changhyun/workspace/cxk858/env/lib/python3.8/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/home/changhyun/workspace/cxk858/env/lib/python3.8/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
/home/changhyun/workspace/cxk858/env/lib/python3.8/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


In [35]:
data_e = data_cleaned.copy()
data_e['EWMA'] = ewma_estim['EWMA']
data_g = data_cleaned.copy()
data_g['GARCH'] = garch_estim['GARCH']
data_r = data_cleaned.copy()
data_r['GJR'] = gjr_estim['GJR']
data_eg = data_cleaned.copy()
data_eg['EWMA'] = ewma_estim['EWMA']
data_eg['GARCH'] = garch_estim['GARCH']
data_er = data_cleaned.copy()
data_er['EWMA'] = ewma_estim['EWMA']
data_er['GJR'] = gjr_estim['GJR']
data_gr = data_cleaned.copy()
data_gr['GJR'] = gjr_estim['GJR']
data_gr['GARCH'] = garch_estim['GARCH']
data_egr = data_cleaned.copy()
data_egr['EWMA'] = ewma_estim['EWMA']
data_egr['GARCH'] = garch_estim['GARCH']
data_egr['GJR'] = gjr_estim['GJR']

In [43]:
Xtrain, ytrain, Xtest, ytest, scaler = data_prep(data_egr, split_index)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2321 entries, 0 to 2320
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Daily_trading_range  2321 non-null   float64
 1   Log_Volume_change    2321 non-null   float64
 2   Daily_log_return     2321 non-null   float64
 3   Index                2321 non-null   float64
 4   gold                 2321 non-null   float64
 5   oil                  2321 non-null   float64
 6   EWMA                 2321 non-null   float64
 7   GARCH                2321 non-null   float64
 8   GJR                  2321 non-null   float64
dtypes: float64(9)
memory usage: 163.3 KB
None
(1834, 22, 9)
(1834, 1)
-----
(443, 22, 9)
(443, 1)


In [44]:
regressor = KerasRegressor(build_fn=create_lstm, epochs=10, verbose=0, batch_size=16)

In [45]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

In [ ]:
tscv = TimeSeriesSplit(n_splits=3)
params = dict(n_units=[10,20,32,64],
              epochs=[50,100],
              batch_size=[8,16,32],
              dropout=[0.1,0.2,0.3,0.4,0.5]
              )
grid = GridSearchCV(estimator=regressor, param_grid=params, verbose=10,cv=tscv)
grid_result = grid.fit(Xtrain,ytrain)